<a href="https://colab.research.google.com/github/BSP22SCM61P/SCM587SP22/blob/main/SearchWithElasticSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import json
import os
import requests
import json
import requests
import ipywidgets as widgets
from IPython.display import clear_output, Markdown, display
from time import time


# Read the original movies.csv file and replace (no genres listed) with NaN values
df = pd.read_csv('movies.csv')
df['genres'] = df['genres'].replace('(no genres listed)', np.NaN)

# Save the cleaned dataframe to a new CSV file
df.to_csv('movies_clean.csv', index=False)

In [3]:
def construct_index(row):
    """Helper function to return all the keywords from the title and genres columns"""
    if pd.isnull(row['genres']):
        genres = None
    else:
        genres = row['genres'].split('|')
         
    return f"{row['title']} {' '.join(genres) if genres else ''}"

In [4]:
# Load cleaned dataset
df = pd.read_csv('movies_clean.csv')

# Construct the index, which includes title + genres
df['search_index'] = df.apply(construct_index, axis=1)


In [5]:
def df2json(df, index_name, id_col, file_loc=' ', file_name='movies.json'):
    """
    Convert the dataframe to a JSON file that is ready to be uploaded to the server

    :param df: the pandas dataframe to be converted
    :param index_name: the name of the Elasticsearch index to be created
    :param id_col: the name of the column in the dataframe to be used as the unique identifier in Elasticsearch
    :param file_loc: the directory where the JSON file will be saved
    :param file_name: the name of the JSON file
    """
    # Remove existing file if it exists
    if file_name in os.listdir(file_loc):
        os.remove(file_loc + file_name)
    
    # Write JSON data to file
    with open(file_loc + file_name, 'a') as f:
        for i, row in df.iterrows():
            d1 = {'index': {'_index': index_name, '_id': f"{row[id_col]}"}}
            d2 = {}
            for col in df.columns:
                if col != id_col:
                    d2[col] = row[col]
            json.dump(d1, f)
            f.write('\n')
            json.dump(d2, f)
            f.write('\n')

df2json(df, 'movies', 'movieId', file_loc='/content/')


In [6]:
def upload_json(file_loc, file_name, endpoint, username, password):
    """
    Bulk upload the JSON data file onto the Elasticsearch domain

    :param file_loc: the directory where the JSON file is saved
    :param file_name: the name of the JSON file
    :param endpoint: the endpoint of the Elasticsearch domain
    :param username: the username used to authenticate with the Elasticsearch domain
    :param password: the password used to authenticate with the Elasticsearch domain
    """
    url = endpoint + '/_bulk'
    with open(file_loc + file_name, 'r') as f:
        r = requests.post(url,
                          data=f,
                          auth=(username, password),
                          headers={'Content-type': 'application/json'})
    return r


# Elasticsearch server configuration
username = 'admin'
password = '123@Admin'
endpoint = 'https://search-project1-afwun7ypkhlhixyp4zbivy6nki.us-east-2.es.amazonaws.com'

# Convert dataframe to JSON and upload to Elasticsearch server
df2json(df, 'movies', 'movieId', file_loc='/content/')
upload_json('/content/', 'movies.json', endpoint, username, password)



<Response [200]>

In [7]:
query = {
    'query': {
        'match': {
            'title': 'jumanji'
        }
    }
}
query = json.dumps(query)

url = endpoint + '/movies/_search'
r = requests.get(url, 
                 auth=(username, password),
                 data=query,
                 headers={'Content-type': 'application/json'})
r.json()

{'took': 1486,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 10.658253,
  'hits': [{'_index': 'movies',
    '_id': '2',
    '_score': 10.658253,
    '_source': {'title': 'Jumanji (1995)',
     'genres': 'Adventure|Children|Fantasy',
     'search_index': 'Jumanji (1995) Adventure Children Fantasy'}}]}}

In [8]:
def search(keyword):
    """Search for the keyword"""
    query = {
        'query': {
            'match_phrase_prefix': {
                'title': keyword
                }
            }
        }
    query = json.dumps(query)

    url = endpoint + '/movies/_search'
    r = requests.post(url, 
                      auth=(username, password), 
                      data=query, 
                      headers={'Content-type': 'application/json'})
    return r.json()


In [9]:
def bold(result, keywords):
    """Bold keywords in the result"""
    keywords = keywords.strip().split(' ')
    for keyword in keywords:
        result = result.replace(keyword, f"**{keyword}**")
        keyword = keyword.title()
        result = result.replace(keyword, f"**{keyword}**")
    return result


In [10]:
def printmd(string):
    """Wrapper function to display markdown"""
    display(Markdown(string))

In [11]:
def search_prefix_index(keywords):
    """Search for the keyword with prefix"""
    if not keywords:
        return None
    keywords = keywords.split(' ')
    if len(keywords) > 1:
        prefix = keywords[-1]
        phrase = ' '.join(keywords[:-1])
        query = {
            'query': {
                'bool': {
                    'must': [
                        {
                            'prefix': {
                                'search_index': prefix
                            }
                        },
                        {
                            'match': {
                                'search_index': {
                                    'query': phrase,
                                    'minimum_should_match': '100%'
                                }
                            }
                        }
                    ]
                }
            }
        }
    else:
        keywords = ' '.join(keywords)
        query = {
            'query': {
                'prefix': {
                    'search_index': keywords
                }
            }
        }

    query = json.dumps(query)

    url = endpoint + '/movies/_search'
    r = requests.post(url, 
                      auth=(username, password), 
                      data=query, 
                      headers={'Content-type': 'application/json'})
    return r.json()

In [12]:
def text_change(change):
    """Widget event when the value changes"""
    global output
    with output:
        clear_output()
        t0 = time()
        results = search(change['new'])
        if results:
            hits = results['hits']['hits']
            print_text = []
            for res in hits[:10]:
                res = res['_source']
                print_text.append(bold(res['title'], change['new']))
            printmd('<br><br>'.join(print_text))
            if hits:
                print(f'\n{round(time() - t0, 4)} seconds')
                
box = widgets.Text(
         value='',
         placeholder='Type something',
         description='Query:',
         disabled=False
     )
output = widgets.Output()
display(box, output)
box.observe(text_change, names='value')

Text(value='', description='Query:', placeholder='Type something')

Output()